# pls work

In [29]:
import pandas as pd
import librosa
import numpy as np
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader


In [30]:
import pandas as pd

# import the csv file
df_labelconvertion = pd.read_csv('fma_metadata/tracks.csv', index_col=1)

# rename columns to actual names
df_labelconvertion.rename(columns={'Unnamed: 0': 'track_id', 'track.7': 'genre_top'}, inplace=True)

# make df of only the important columns
df_labelconvertion = df_labelconvertion[['track_id', 'genre_top']]

# set all NA values to other 
df_labelconvertion['genre_top'] = df_labelconvertion['genre_top'].fillna('Other')

# reset index from albums to normal index
df_labelconvertion = df_labelconvertion.reset_index(drop=True)

# make all values in the track_id column strings
df_labelconvertion['track_id'] = df_labelconvertion['track_id'].astype(str)

# remove first two rows
df_labelconvertion = df_labelconvertion.iloc[2:]


df_labelconvertion


C:\Users\rama0\AppData\Local\Temp\ipykernel_31608\2505249569.py:4: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df_labelconvertion = pd.read_csv('fma_metadata/tracks.csv', index_col=1)


,track_id,genre_top
2,2,Hip-Hop
3,3,Hip-Hop
4,5,Hip-Hop
5,10,Pop
6,20,Other
...,...,...
106571,155316,Rock
106572,155317,Rock
106573,155318,Rock
106574,155319,Rock


In [31]:
df_labelnumbers = pd.read_csv('fma_metadata/labelnumber.csv')

df_labelnumbers.rename(columns={'Unnamed: 0': 'genre', '0': 'number'}, inplace=True)

df_labelnumbers

,genre,number
0,Hip-Hop,0
1,Pop,1
2,Other,2
3,Rock,3
4,Experimental,4
5,Folk,5
6,Jazz,6
7,Electronic,7
8,Spoken,8
9,International,9


In [32]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import torch

class SpectrogramDataset(Dataset):
    def __init__(self, file_paths, sr=44100, n_fft=1024, hop_length=512, window='hann', transform=None):
        self.sr = sr
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.window = window
        self.transform = transform
        self.file_paths = file_paths
                    
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        y, sr = librosa.load(file_path, sr=self.sr, mono = True)
        y = y[:10 * self.sr]
        D = librosa.stft(y, n_fft=self.n_fft, hop_length=self.hop_length, window=self.window)
        D_real = np.real(D)
        D_imag = np.imag(D)
        D_real = torch.tensor(D_real, dtype=torch.float32)
        D_imag = torch.tensor(D_imag, dtype=torch.float32)
        
        # concat real and imaginary parts to 2 channels
        D = torch.stack((D_real, D_imag), dim=0)
        # get the label from the file name and convert to genre
        track_id = os.path.basename(file_path).replace('.mp3', '').lstrip('0')
        # try to get the label from the dataframe
        try:
            # label = df_labelconvertion.query('track_id == @track_id')['genre_top'].values[0]
            label = df_labelconvertion.query('track_id == @track_id')['genre_top'].values[0]
            label = torch.tensor(df_labelnumbers.query('genre == @label')['number'].values[0])
        except IndexError:
            # if not found, set label to 'Other'
            print(f"Track ID: {track_id} not found in dataframe")
            label = 'Other'
        
        return D, label

In [33]:
# create dataloader
batch_size = 32
num_workers = 4
file_directory = 'fma_small'
# train test split
from sklearn.model_selection import train_test_split
# get all file paths
file_paths = []
for root, _, files in os.walk(file_directory):
    for file in files:
        if file.endswith('.mp3'):
            file_paths.append(os.path.join(root, file))
            
# split the file paths into train and test
train_paths, test_paths = train_test_split(file_paths, test_size=0.2, random_state=42)

# # find if any song is shorter than 10 seconds or is corrupted
# for path in train_paths:
#     try:
#         y, sr = librosa.load(path, sr=44100, mono=True)
#         if len(y) < 10 * sr:
#             print(f"File {path} is shorter than 10 seconds")
#             train_paths.remove(path)
#     except Exception as e:
#         print(f"File {path} is corrupted: {e}")
#         train_paths.remove(path)
        
# for path in test_paths:
#     try:
#         y, sr = librosa.load(path, sr=44100, mono=True)
#         if len(y) < 10 * sr:
#             print(f"File {path} is shorter than 10 seconds")
#             test_paths.remove(path)
#     except Exception as e:
#         print(f"File {path} is corrupted: {e}")
#         test_paths.remove(path)

train_dataset = SpectrogramDataset(train_paths)
test_dataset = SpectrogramDataset(test_paths)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [34]:
# check the shape of the data
for batch in train_dataloader:
    print(batch[0].shape)
    print(batch[1].shape)
    break

torch.Size([32, 2, 513, 862])
torch.Size([32])


In [35]:
import cvae_actual_i_think
import importlib
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger

importlib.reload(cvae_actual_i_think) # Reload the cvae module to ensure it's up to date

model = cvae_actual_i_think.ConvCVAEPL()

# print(hasattr(model, 'forward'))

logger = CSVLogger("logs", name="train_10_epochs")

# Initialize the PyTorch Lightning Trainer
trainer = Trainer(max_epochs=5, accelerator="auto", logger=logger) #TODO


# Train the model
import time

start_time = time.time()
print("Training started")

trainer.fit(model, train_dataloader, test_dataloader)
print("Training finished")
end_time = time.time()
print(f"Training time: {end_time - start_time} seconds")

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type     | Params | Mode 
-------------------------------------------
0 | model | ConvCVAE | 354 M  | train
-------------------------------------------
354 M     Trainable params
0         Non-trainable params
354 M     Total params
1,416.385 Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


Training started
Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\rama0\OneDrive\Skrivbord\Skola\TIF360_proj\TIF360ProjectD\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
C:\Users\rama0\AppData\Local\Temp\ipykernel_31608\30593426.py:20: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=self.sr, mono = True)
c:\Users\rama0\OneDrive\Skrivbord\Skola\TIF360_proj\TIF360ProjectD\venv\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


NoBackendError: 